In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage.feature import local_binary_pattern
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [2]:
def load_and_preprocess_image(image_path):
    # Read the image
    img = cv2.imread(image_path)
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Apply Otsu's thresholding
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour = max(contours, key=cv2.contourArea)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB), gray, binary, contour


# Resample boundary to 256 points
def resample_points(contour, n_points=256):
    contour = contour.squeeze()  # Ensure contour is a 2D array
    if contour.ndim == 3:
        contour = contour.reshape(-1, 2)  # Reshape to (n, 2) if it's (n, 1, 2)
    elif contour.ndim == 1:
        contour = contour.reshape(-1, 2)  # Reshape to (n, 2) if it's (2n,)
    
    distances = np.cumsum(np.sqrt(np.sum(np.diff(contour, axis=0)**2, axis=1)))
    distances = np.insert(distances, 0, 0)
    total_length = distances[-1]
    spacing = np.linspace(0, total_length, n_points)
    return np.array([np.interp(spacing, distances, contour[:, i]) for i in [0, 1]]).T.astype(np.int32)

In [3]:
def mtd_feature(contour, N=2048, M=5):
    # Sample the contour evenly into N points
    sampled_points = resample_points(contour, N)
    Ts = int(np.floor(np.log2(N/2)))
    
    # Initialize feature arrays
    alpha = np.zeros((N, Ts))
    beta = np.zeros((N, Ts))
    gamma = np.zeros((N, Ts))

    for i in range(N):
        Pi = sampled_points[i]
        for k in range(1, Ts + 1):
            d_k = 2**(k-1)
            Pi_plus_d_k = sampled_points[(i + d_k) % N]
            Pi_minus_d_k = sampled_points[(i - d_k - 1) % N]

            # Calculate signed area
            t_sa = 0.5 * np.linalg.det(np.array([
                [Pi_minus_d_k[0], Pi_minus_d_k[1], 1],
                [Pi[0], Pi[1], 1],
                [Pi_plus_d_k[0], Pi_plus_d_k[1], 1]
            ]))
            alpha[i, k-1] = np.abs(t_sa)
            beta[i, k-1] = 1 if t_sa >= 0 else 0

            # Calculate center distance
            centroid = (Pi_minus_d_k + Pi + Pi_plus_d_k) / 3
            gamma[i, k-1] = np.sqrt((Pi[0] - centroid[0])**2 + (Pi[1] - centroid[1])**2)

    # Normalize alpha and gamma features column-wise to achieve scale invariance.
    for k in range(Ts):
        max_alpha = np.max(np.abs(alpha[:, k]))
        if max_alpha != 0:
            alpha[:, k] /= max_alpha
        max_gamma = np.max(np.abs(gamma[:, k]))
        if max_gamma != 0:
            gamma[:, k] /= max_gamma
    
    # Apply discrete Fourier transform (DFT) on each feature (each column) along the contour axis.
    # We take the amplitude (absolute values) of the first M coefficients.
    alpha_fft = np.zeros((M, Ts))
    beta_fft = np.zeros((M, Ts))
    gamma_fft = np.zeros((M, Ts))
    
    for k in range(Ts):
        fft_alpha = np.fft.fft(alpha[:, k])
        fft_beta = np.fft.fft(beta[:, k])
        fft_gamma = np.fft.fft(gamma[:, k])
        # Retain the first M low-frequency components
        alpha_fft[:, k] = np.abs(fft_alpha[:M])
        beta_fft[:, k] = np.abs(fft_beta[:M])
        gamma_fft[:, k] = np.abs(fft_gamma[:M])
    
    # Concatenate the features (flatten each feature type and then concatenate)
    # Final feature dimension: 3 * (M * Ts)
    mtd_descriptor = np.concatenate((alpha_fft.flatten(),
                                     beta_fft.flatten(),
                                     gamma_fft.flatten()))
    return mtd_descriptor

# Example usage:
# mtd_features = mtd_feature(query_contour)
# print(mtd_features)
# print(mtd_features.shape) 

In [4]:
def lbp_hf_feature(gray_image, P=24, R=3):
    # Calculate LBP
    lbp_image = local_binary_pattern(gray_image, P, R, method='uniform')
    
    # Helper functions
    def code_to_binary_tuple(code, P):
        code = int(code)
        return tuple((code >> np.arange(P)) & 1)
    
    def is_uniform(binary_tuple):
        transitions = sum(binary_tuple[i] != binary_tuple[(i+1) % len(binary_tuple)] for i in range(len(binary_tuple)))
        return transitions <= 2
    
    def rotation_index(binary_tuple):
        best = binary_tuple
        best_shift = 0
        for s in range(1, len(binary_tuple)):
            shifted = binary_tuple[s:] + binary_tuple[:s]
            if shifted < best:
                best = shifted
                best_shift = s
        return best_shift
    
    # Build a lookup dictionary for unique codes
    unique_codes = np.unique(lbp_image)
    uniform_map = {}
    for code in unique_codes:
        b = code_to_binary_tuple(code, P)
        if is_uniform(b):
            n = sum(b)
            r = rotation_index(b)
            uniform_map[int(code)] = ('uniform', n, r)
        else:
            uniform_map[int(code)] = ('nonuniform', None, None)
    
    # Initialize histogram for uniform patterns (for n = 1,...,P-1)
    H_uniform = {n: np.zeros(P, dtype=int) for n in range(1, P)}
    count_zero = 0
    count_allone = 0
    count_nonuniform = 0
    
    # Process each pixel and accumulate histogram counts
    for code in lbp_image.ravel():
        typ, n, r = uniform_map[int(code)]
        if typ == 'uniform':
            if n == 0:
                count_zero += 1
            elif n == P:
                count_allone += 1
            else:
                H_uniform[n][r] += 1
        else:
            count_nonuniform += 1
    
    # For each uniform pattern row, compute the FFT and keep first (P//2+1) coefficients
    lbp_hf_features = []
    num_coeff = P // 2 + 1
    for n in range(1, P):
        hist_row = H_uniform[n]
        fft_coeff = np.fft.fft(hist_row)
        lbp_hf_features.extend(np.abs(fft_coeff[:num_coeff]))
    
    # Append the extra 3 bins
    extra_features = np.array([count_zero, count_allone, count_nonuniform], dtype=np.float32)
    
    feature_vector = np.concatenate((np.array(lbp_hf_features, dtype=np.float32), extra_features))
    
    # Normalize the final feature vector
    if feature_vector.sum() > 0:
        feature_vector = feature_vector / feature_vector.sum()
    
    return feature_vector

# Example usage:
# lbp_hf_features = lbp_hf_feature(query_img_gray)
# print(lbp_hf_features)
# print(lbp_hf_features.shape)

In [5]:
def combine_features(mtd_features, lbp_hf_features, w4=0.5, w5=0.5):
    combined_features = np.concatenate([mtd_features * w4, lbp_hf_features * w5])
    return combined_features

# Example usage:
# combined_features = combine_features(mtd_features, lbp_hf_features)

In [6]:
def train_and_evaluate(X_train, X_test, y_train, y_test, model):
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    # print(f"Accuracy: {accuracy:.4f}")
    # print(f"F1-score: {f1:.4f}")
    return accuracy, f1

In [7]:
def full_pipeline(dataset_root, n_iter, model, done):
    N_POINTS = 2048    
    M_COEFFS = 5       
    LBP_RADIUS = 3     
    LBP_POINTS = 24    
    
    # Load features and labels
    features, labels = [], []
    if(done == 0):
        for class_id in range(1, 101):  # t1 to t100 for CVIP100
            class_dir = os.path.join(dataset_root, f't{class_id}')  
            for img_file in sorted(os.listdir(class_dir)):  
                img_path = os.path.join(class_dir, img_file)
                _, gray, _, contour = load_and_preprocess_image(img_path)
                
                # Extract mtd and LBP-HF feature
                mtd = mtd_feature(contour, N=N_POINTS, M=M_COEFFS)
                lbp = lbp_hf_feature(gray, P=LBP_POINTS, R=LBP_RADIUS)
                
                features.append(combine_features(mtd, lbp))
                # features.append(mtd)
                # features.append(lbp)
                labels.append(class_id-1)
        np.save('CVIP100_concat_features.npy', features)
        np.save('CVIP100_labels.npy', labels)
    else:
        features = np.load('CVIP100_concat_features.npy')
        
        labels = np.load('CVIP100_labels.npy')
    scaler = StandardScaler()
    features = scaler.fit_transform(features)
    acc_score_list = []
    f1_score_list = []
    def iteration(n_iter, features, labels, model):
        for i in range(n_iter):
            # train and test dataset split
            X_train, X_test, y_train, y_test = train_test_split(
                features, labels, test_size=0.3, stratify=None, random_state=i)
            acc, f1= train_and_evaluate(X_train, X_test, y_train, y_test, model)
            acc_score_list.append(acc)
            f1_score_list.append(f1)
        return np.mean(acc_score_list), np.mean(f1_score_list)

    return iteration(n_iter, features, labels, model)


In [8]:
# Example usage:
n_iter = 20
dataset_root = r'dataset/CVIP100 Leaf Dataset'        
# KNN
accuracy, f1 = full_pipeline(dataset_root, n_iter, KNeighborsClassifier(n_neighbors=1),0)
print(f"KNN Result Final Accuracy: {accuracy:.4f} Final F1-score: {f1:.4f}")

# SVM
accuracy, f1 = full_pipeline(dataset_root, n_iter, SVC(kernel='rbf', C=10, gamma=0.001),1)
print(f"SVM Result Final Accuracy: {accuracy:.4f} Final F1-score: {f1:.4f}")

# Naive Bayes
accuracy, f1 = full_pipeline(dataset_root, n_iter, GaussianNB(),1)
print(f"Gaussian Naive Bayes Result Final Accuracy: {accuracy:.4f} Final F1-score: {f1:.4f}")

accuracy, f1 = full_pipeline(dataset_root, n_iter, LogisticRegression(max_iter=1000),1)
print(f"Logistic Regression Result Final Accuracy: {accuracy:.4f} Final F1-score: {f1:.4f}")

accuracy, f1 = full_pipeline(dataset_root, n_iter, DecisionTreeClassifier(),1)
print(f"Decision Tree Result Final Accuracy: {accuracy:.4f} Final F1-score: {f1:.4f}")


KNN Result Final Accuracy: 0.9354 Final F1-score: 0.9352
SVM Result Final Accuracy: 0.9618 Final F1-score: 0.9617
Gaussian Naive Bayes Result Final Accuracy: 0.8975 Final F1-score: 0.8983
Logistic Regression Result Final Accuracy: 0.9644 Final F1-score: 0.9647
Decision Tree Result Final Accuracy: 0.6056 Final F1-score: 0.5998


In [9]:
def full_pipeline_swedish(dataset_root, n_iter, model,done):
    N_POINTS = 2048    
    M_COEFFS = 5       
    LBP_RADIUS = 3     
    LBP_POINTS = 24    
    
    # Load features and labels
    features, labels = [], []
    if(done == 0):
        for class_id in range(1, 16):  # t1 to t100 for CVIP100
            class_dir = os.path.join(dataset_root, f'leaf{class_id}')  
            for img_file in sorted(os.listdir(class_dir)):  
                img_path = os.path.join(class_dir, img_file)
                _, gray, _, contour = load_and_preprocess_image(img_path)
                
                # Extract mtd and LBP-HF feature
                mtd = mtd_feature(contour, N=N_POINTS, M=M_COEFFS)
                lbp = lbp_hf_feature(gray, P=LBP_POINTS, R=LBP_RADIUS)
                
                features.append(combine_features(mtd, lbp))
                # features.append(mtd)
                # features.append(lbp)
                labels.append(class_id-1)
        np.save('swedish_concat_features.npy', features)
        np.save('swedish_labels.npy', labels)
    else:
        features = np.load('swedish_concat_features.npy')
        labels = np.load('swedish_labels.npy')
    scaler = StandardScaler()
    features = scaler.fit_transform(features)
    acc_score_list = []
    f1_score_list = []
    def iteration(n_iter, features, labels, model):
        for i in range(n_iter):
            # train and test dataset split
            X_train, X_test, y_train, y_test = train_test_split(
                features, labels, test_size=0.3, stratify=None, random_state=i)
            acc, f1= train_and_evaluate(X_train, X_test, y_train, y_test, model)
            acc_score_list.append(acc)
            f1_score_list.append(f1)
        return np.mean(acc_score_list), np.mean(f1_score_list)

    return iteration(n_iter, features, labels, model)


In [10]:
# Example usage:
n_iter = 20
dataset_root = r'dataset/Swedish Leaf Dataset'        
# KNN
accuracy, f1 = full_pipeline_swedish(dataset_root, n_iter, KNeighborsClassifier(n_neighbors=1),0)
print(f"KNN Result Final Accuracy: {accuracy:.4f} Final F1-score: {f1:.4f}")

# SVM
accuracy, f1 = full_pipeline_swedish(dataset_root, n_iter, SVC(kernel='rbf', C=10, gamma=0.001),1)
print(f"SVM Result Final Accuracy: {accuracy:.4f} Final F1-score: {f1:.4f}")

# Naive Bayes
accuracy, f1 = full_pipeline_swedish(dataset_root, n_iter, GaussianNB(),1)
print(f"Gaussian Naive Bayes Result Final Accuracy: {accuracy:.4f} Final F1-score: {f1:.4f}")

accuracy, f1 = full_pipeline_swedish(dataset_root, n_iter, LogisticRegression(max_iter=1000),1)
print(f"Logistic Regression Result Final Accuracy: {accuracy:.4f} Final F1-score: {f1:.4f}")

accuracy, f1 = full_pipeline_swedish(dataset_root, n_iter, DecisionTreeClassifier(),1)
print(f"Decision Tree Result Final Accuracy: {accuracy:.4f} Final F1-score: {f1:.4f}")


KNN Result Final Accuracy: 0.9682 Final F1-score: 0.9682
SVM Result Final Accuracy: 0.9864 Final F1-score: 0.9864
Gaussian Naive Bayes Result Final Accuracy: 0.9330 Final F1-score: 0.9346
Logistic Regression Result Final Accuracy: 0.9874 Final F1-score: 0.9874
Decision Tree Result Final Accuracy: 0.8879 Final F1-score: 0.8886
